In [2]:
#connect to google drive where I have put my training and testing images
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Facial\ Recognition/py_utils/py_utils/*.* /content

In [0]:
#get py_utils
!cp /content/gdrive/My\ Drive/py_utils/*.* /content

In [4]:
#i have retrograded back to tensorflow version 1.x
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
#install vggface from git
!pip install git+https://github.com/rcmalli/keras-vggface.git --quiet

!pip install ipywidgets --quiet

In [7]:
import tensorflow
print(tensorflow.__version__)

1.15.2


# Importing the libraries

In [8]:
import os, sys
import glob
import shutil
import zipfile
import numpy as np
import math
import tarfile
import re
import pickle
import requests
from tqdm import tqdm_notebook as tqdm
from numpy.random import random, permutation
from matplotlib import pyplot as plt
from PIL import Image
import time
import cv2

from keras import backend as K
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Lambda
from keras.utils.data_utils import get_file
from itertools import islice
from matplotlib import rcParams
import random
from PIL import ImageOps, ImageFont, ImageDraw 
import dlib
from skimage import io, transform
import tensorflow as tf
import ipywidgets as widgets
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras_vggface import utils

from myutils import *

Using TensorFlow backend.


# Creating paths for original, cropped images



In [0]:
train_org_path = "/content/gdrive/My Drive/Colabstuff/train/"
test_org_path = "/content/gdrive/My Drive/Colabstuff/test/"
train_cropped_path = "/content/gdrive/My Drive/Colabstuff/pres_images/train_cropped/"
test_cropped_path = "/content/gdrive/My Drive/Colabstuff/test_cropped/"

# Detect the face of the images from the original downloaded images

In [0]:
def detect_faces(image):
    
    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()

    # Run detector and get bounding boxes of the faces on image.
    detected_faces = face_detector(image, 1)
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]

    return face_frames

# Created a cropped image

In [0]:
def create_cropped_images(original_path, cropped_path):
  for f in os.listdir(original_path):
    print(f)
    #creating a folder as in original
    name_cropped_dir = cropped_path+f+'/'
    make_path(name_cropped_dir)
    for r, d, fil_list in os.walk(original_path+f):
      for fname in fil_list:
        image = cv2.imread(original_path+f+'/'+fname)
        
        detected_faces = detect_faces(image)
        if (len(detected_faces)):
          face = Image.fromarray(image).crop(detected_faces[0])
          face.save(name_cropped_dir+fname)

In [0]:
def create_cropped_images(original_path, cropped_path):
  for f in os.listdir(original_path):
    if (f=='.ipynb_checkpoints'):
        #going to the cropped path
        print('')
    else:
      #for r, d, fil_list in os.walk(original_path):
      image = cv2.imread(original_path+'/'+f)
      
      detected_faces = detect_faces(image)
      if (len(detected_faces)):
        face = Image.fromarray(image).crop(detected_faces[0])
        face.save(cropped_path+f)

# Creating a cropped version of the train images


In [0]:
#create cropped images of the train
create_cropped_images(train_org_path,train_cropped_path)

# Creating a cropped version of the test images

In [0]:
#create cropped images of the test
create_cropped_images(test_org_path,test_cropped_path)

### Model selection

In [35]:
senet_model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


104947712/104944616 [==============================] - 4s 0us/step


In [36]:
senet_model.summary()

Model: "vggface_senet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
____________________________________________________________________________________

In [0]:
#customize the model. Add a flatten layer and then add fully connected layer with 128 units. Use L2 normalization
def fr_model():
  
  last_layer = senet_model.get_layer('avg_pool').output
  X = Flatten(name='flatten')(last_layer)
  X = Dense(128, name='dense_layer')(X)
  # L2 normalization
  out = Lambda(lambda  x: K.l2_normalize(x,axis=1))(X)

  # Create model instance
  model = Model(senet_model.input, out, name='fr_model')
  return model


In [39]:
model = fr_model()
model.summary()

Model: "fr_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
___________________________________________________________________________________________

### Image encoding

In [0]:
def img_to_encoding(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = utils.preprocess_input(x, version=2)
    embedding = model.predict(x)
    return embedding

#Distance Calculation

In [43]:

#Train data
encodes_train = []
selected_path="/content/gdrive/My Drive/Colabstuff/pres_images/train_cropped"
for f in os.listdir(selected_path):
  print(f)
  if (f=='.ipynb_checkpoints'):
        #going to the cropped path
        print('')
  else:
      encoding = img_to_encoding(selected_path+'/'+f, model)
      encodes_train.append(encoding)
  
#Test data
encodes_test = []
selected_path="/content/gdrive/My Drive/Colabstuff/test_cropped"
for f in os.listdir(selected_path):
  print(f)
  if (f=='.ipynb_checkpoints'):
        #going to the cropped path
        print('')
  else:
      encoding = img_to_encoding(selected_path+'/'+f, model)
      encodes_test.append(encoding)

cameron-diaz-train1.PNG

cameron-diaz-train4.PNG
cameron-diaz-train2.PNG
cameron-diaz-train5.PNG
cameron-diaz-train3.PNG
stylecameron-diaz4.jpg
stylecameron-diaz1.jpg
stylerachael.jpg
styleellen_barkin.jpg
styleca.jpg
stylecameron-diaz2.jpg
styleangelina.jpg
stylecameron-diaz3.jpg


In [0]:
dist=[]
for i in range(len(encodes_train)):
  for j in range(len(encodes_test)):
      eucle_dist = np.linalg.norm(encodes_train[i]-encodes_test[j])
      dist.append([i,j,eucle_dist])

In [0]:
#create a dataframe of distances between images
import pandas as pd
df = pd.DataFrame(dist,columns=['train','test','dist'])
eucle_dis_matrxi=pd.crosstab(index=df['train'],columns=df['test'],values=df['dist'],aggfunc='sum').reset_index()
eucle_dis_matrix_clean=eucle_dis_matrxi.loc[:,~eucle_dis_matrxi.columns.isin(['train'])]
eucle_dis_matrix_clean.columns=['diaz1','diaz2','diaz3','diaz4','angelina','diaz_orig','aniston','ellen_barkin']

In [46]:
#plot the distance map 
import seaborn as sns
color_map = sns.light_palette("green", as_cmap=True)
eucle_dis_matrix_clean.style.background_gradient(cmap=color_map)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,diaz1,diaz2,diaz3,diaz4,angelina,diaz_orig,aniston,ellen_barkin
0,1.039428,1.034720,1.235228,1.054094,0.671746,1.026548,0.991751,1.089564
1,0.994262,0.956474,1.142204,0.997606,0.485124,0.930685,1.009184,0.987840
2,1.089842,0.977478,1.149884,0.873930,0.547544,1.018260,1.086834,0.859464
3,1.117448,0.939167,1.212051,0.797134,0.483966,0.929232,1.133260,0.961614
4,1.076155,0.982658,1.169882,0.883781,0.742445,1.029060,1.088982,0.819446


In [47]:
eucle_dis_matrix_clean.describe().T['mean'].mean()

0.9653994299471378